## Test cases

We have provided some test cases for you to help verify your implementation.

In [596]:

import numpy as np

class MDPAgent(object):
    def __init__(self):
        pass
    
    def getMDPEnv(self, is_bad_side):
        N = len(is_bad_side)
        is_bad_side = np.array(is_bad_side)
        good_sides = (np.where(is_bad_side == 0)[0] + 1).tolist()
        states =  good_sides
        is_all_zero = np.all((is_bad_side == 0))
        k = []
        for i in states:
            for j in states:
                k += [i + j]
        states = np.unique(sorted( states + k )).tolist()
        good_sides_second_roll = states

        k = []
        for i in states:
            for j in states:
                k += [i + j]
        
        states = np.unique(sorted([0] + states + k + [99])).tolist()
    
        num_state = len(states)

        transition_prob = np.zeros([2, num_state, num_state])
        reward_prob = np.zeros([2, num_state, num_state])
        # Roll Action transition
        for i in range(len(states)):
            terminal_prob  = 1
            sum_prob = 0
            for j in range(len(states)):
                if (states[j]-states[i] in good_sides):
                    transition_prob[0][i][j] = 1/N
                    reward_prob[0][i][j] = states[j]-states[i]
                    sum_prob += 1/N            
                    
            transition_prob[0][i][-1] = 1 - sum_prob
            reward_prob[0][i][-1] =  -states[i] if not is_all_zero else 0 
        
        reward_prob[0][-1][-1] =  0
        # Quit action transition
        transition_prob[1] = np.identity(num_state)   
        reward_prob[1] = np.zeros(num_state)
        return transition_prob, reward_prob

        
    def valueIteration(self, transition_prob, reward_prob):
        num_state = len(transition_prob[1])

        v = np.empty(num_state)
        
        reward_transition_prob = np.zeros([2, num_state])
        reward_transition_prob[0] = (reward_prob[0] * transition_prob[0]).sum(1)
        reward_transition_prob[1] = (reward_prob[1] * transition_prob[1]).sum(1)
        
        policy = np.empty((2, num_state))
        i = -1
        while True:
            i += 1
            old_v = v.copy()
            policy[0] = reward_transition_prob[0] + transition_prob[0].dot(v)
            policy[1] = reward_transition_prob[1] + transition_prob[1].dot(v)

            v = policy.max(axis=0)
            
            delta =  (v - old_v).max() - (v - old_v).min()
            if (delta < 0.00000001 and i != 0) or (i > 200000000000):
                return v
        

    def solve(self, is_bad_side):
        """Implement the agent"""
        is_bad_side = np.array(is_bad_side)

        # Defining MDP
        transition_prob, reward_prob = self.getMDPEnv(is_bad_side)
        value_prob  = self.valueIteration(transition_prob, reward_prob)
        print(value_prob[0])
        return value_prob[0]


In [597]:
agent = MDPAgent()
agent.solve([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

81.31357923357321


81.31357923357321